In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input, Output, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment, ResourceConfiguration
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction
from azure.ai.ml.entities import AmlCompute
from dotenv import load_dotenv, find_dotenv
import os
import mlflow

In [2]:
# Initialize ML Client
load_dotenv(find_dotenv(), override=True)

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

In [3]:
# Create Environment
custom_env_name = "mm-remote-env-py37"

try:
    # let's see if the environment already exists
    env = ml_client.environments.get(custom_env_name, label="latest")
    print(f"You already have a environment named {custom_env_name}, we'll reuse it as is.")

except Exception:
    print("Creating a new environment...")
    custom_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for many models",
        conda_file="../environment/remote_env_conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    )
    custom_job_env = ml_client.environments.create_or_update(custom_job_env)

    print(f"Environment created: {custom_job_env.name}")

ml_client.environments.get(custom_env_name, label="latest")

You already have a environment named mm-remote-env-py37, we'll reuse it as is.


Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'mm-remote-env-py37', 'description': 'Custom environment for many models', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourceGroups/many-models-rg/providers/Microsoft.MachineLearningServices/workspaces/mm-aml-wksp/environments/mm-remote-env-py37/versions/2', 'Resource__source_path': None, 'base_path': '/home/zacksoenen/Projects/many-models-azureml/demo_notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f127f9d1b50>, 'serialize': <msrest.serialization.Serializer object at 0x7f127f9d1e50>, 'version': '2', 'latest_version': None, 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.7.6', 'pip', {'pip': ['azureml-mlflow', 'mlflow', 'mltable>=1.2.0', 'azureml-dataset-runtime[pandas,fuse]', 'azureml-teleme

In [4]:
# Load data preparation component
partition_data = load_component(source="../src/components/partition_data/partition_data.yaml")

In [5]:
# Acess data asset
data_name = "oj-sim-sales-train"
train_data_asset = ml_client.data.get(data_name, label='latest')
print(train_data_asset.path)

azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/eabbea2d8b14c97a278f2f297b333403/train_subset.csv


In [6]:
# Define compute target (created in setup notebook)
cpu_compute_target = "mm-cpu-cluster"

In [10]:
# Declare parallel job, with a run_function task
many_model_training_with_partition_keys = parallel_run_function(
    name="distributed_training",
    display_name="Many Model Training",
    description="parallel job to train many models with partition_keys on mltable input",
    inputs=dict(
        data_source=Input(
            type=AssetTypes.MLTABLE,
            description="Input mltable with predefined partition format.",
            mode=InputOutputModes.DIRECT,  # [Important] To use 'partition_keys', input MLTable is required to use 'direct' mode.
        ),
        drop_cols=Input(
            type="string",
            description="Columns need to be dropped before training. Split by comma.",
        ),
        target_col=Input(
            type="string",
            description="The column name for label of the input data.",
        ),
        date_col=Input(
            type="string",
            description="The column name for datatime.",
        ),
    ),
    outputs=dict(
        model_folder=Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT,
        )
    ),
    input_data="${{inputs.data_source}}",  # Define which input data will be splitted into mini-batches
    partition_keys=["Store","Brand"],  # Use 'partition_keys' as the data division method. This method requires MLTable input with partition setting pre-defined in MLTable artifact.
    instance_count=6,  # Use X nodes from compute cluster to run this parallel job.
    max_concurrency_per_instance=2,  # Create 2 worker processors in each compute node to execute mini-batches.
    error_threshold=-1,  # Monitor the failures of item processed by the gap between mini-batch input count and returns. 'Many model training' scenario doesn't fit this setting and '-1' means ignore counting failure items by mini-batch returns.
    mini_batch_error_threshold=5,  # Monitor the failed mini-batch by exception, time out, or null return. When failed mini-batch count is higher than this setting, the parallel job will be marked as 'failed'.
    retry_settings=dict(
        max_retries=1,  # Define how many retries when mini-batch execution is failed by exception, time out, or null return.
        timeout=60,  # Define the timeout in second for each mini-batch execution.
    ),
    logging_level="DEBUG", # DEBUG, INFO, WARNING, ERROR, ETC
    environment_variables={
        "AZUREML_PARALLEL_EXAMPLE": "1a_sdk",
    },
    task=RunFunction(
        code="../src/",
        entry_script="parallel_train.py",
        environment=ml_client.environments.get(custom_env_name, label="latest"),
        program_arguments="--drop_cols ${{inputs.drop_cols}} "  # Passthrough input parameters into parallel_train script.
        "--target_col ${{inputs.target_col}} "
        "--date_col ${{inputs.date_col}} "
        "--model_folder ${{outputs.model_folder}} ",
    ),
)

In [11]:
# Build Pipeline
# Declare the overall input of the job.
input_oj_data = Input(
    path=train_data_asset.path,
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.RO_MOUNT,
)

# Declare pipeline structure.
@pipeline(display_name="parallel train pipeline-nolagfinal")
def parallel_training_pipeline(pipeline_input_data):
    
    # Declare 1st data partition command job.
    partition_job = partition_data(
        data_source=pipeline_input_data,
        partition_keys="Store,Brand",
    )

    # Declare 2nd parallel model training job.
    parallel_train = many_model_training_with_partition_keys(
        data_source=partition_job.outputs.tabular_output_data,
        drop_cols="Advert,Store,Brand",
        target_col="Revenue",
        date_col="WeekStarting",
    )
    
    # User could override parallel job run-level property when invoke that parallel job/component in pipeline.
    # parallel_train.resources.instance_count = 5
    # parallel_train.max_concurrency_per_instance = 2
    # parallel_train.mini_batch_error_threshold = 10

# Create pipeline instance
training_pipeline = parallel_training_pipeline(pipeline_input_data=input_oj_data,)

# Set pipeline level compute
training_pipeline.settings.default_compute = cpu_compute_target
print(training_pipeline)

display_name: parallel train pipeline-nolagfinal
type: pipeline
inputs:
  pipeline_input_data:
    mode: ro_mount
    type: uri_file
    path: azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/eabbea2d8b14c97a278f2f297b333403/train_subset.csv
jobs:
  partition_job:
    type: command
    inputs:
      data_source:
        path: ${{parent.inputs.pipeline_input_data}}
      partition_keys: Store,Brand
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: partition_data
      version: '1'
      display_name: Partition data by keys
      type: command
      inputs:
        data_source:
          type: uri_file
        partition_keys:
          type: string
      outputs:
        tabular_output_data:
          type: mltable
      command: python partition_data.py --data_source ${{inputs.data_source}} --partition_keys
    

In [12]:
# Submit training pipeline
pipeline_job = ml_client.jobs.create_or_update(
    training_pipeline,
    experiment_name="many-models-parallel-training-job-mlflow-full",
)


Uploading src (0.01 MBs): 100%|██████████| 11967/11967 [00:00<00:00, 62756.06it/s]




In [6]:
# Analyze results using mlflow

azureml_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_tracking_uri)

EXPIREMENTS = ["many-models-parallel-training-job-mlflow-full"]

mlflow.search_runs(experiment_names= EXPIREMENTS,
                   filter_string="tags.store = '1013' and \
                                  metrics.test_mape < 30 and \
                                  tags.brand = 'tropicana' "
                )

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_mse,metrics.training_mae,metrics.test_mape,metrics.training_rmse,...,params.validation_fraction,params.n_estimators,tags.mlflow.rootRunId,tags.brand,tags.estimator_name,tags.estimator_class,tags.mlflow.user,tags.mlflow.runName,tags.store,tags.mlflow.parentRunId
0,146583e2-3943-4175-9a53-cc03b080565f,e6a75da7-2df2-4211-8f15-b4e8d9ee8e23,FINISHED,,2024-01-04 19:48:36.092000+00:00,2024-01-04 19:48:45.645000+00:00,1.534044e+06,989.14651,21.694092,1238.565122,...,0.1,100,red_planet_0b4kbtckls,tropicana,GradientBoostingRegressor,sklearn.ensemble.gradient_boosting.GradientBoo...,Zack Soenen,red_pot_7792w52c,1013,2bccc81e-a97c-4704-bb06-e889701172e2
1,1553b5c6-3ebe-4f6c-b3c1-60ec937a34ed,e6a75da7-2df2-4211-8f15-b4e8d9ee8e23,FINISHED,,2024-01-04 20:47:40.063000+00:00,2024-01-04 20:47:48.519000+00:00,1.534044e+06,989.14651,21.694092,1238.565122,...,0.1,100,olden_rice_ft8kfvwq0k,tropicana,GradientBoostingRegressor,sklearn.ensemble.gradient_boosting.GradientBoo...,Zack Soenen,tropicana_1013_job,1013,45c31c6c-f651-4488-878c-510b6e2d2590
2,16cdf992-82c3-44f0-a61f-e7ddd580e606,e6a75da7-2df2-4211-8f15-b4e8d9ee8e23,FINISHED,,2024-01-05 22:45:34.272000+00:00,2024-01-05 22:45:43.420000+00:00,1.534044e+06,989.14651,21.694092,1238.565122,...,0.1,100,sleepy_neck_0qdqnyryf0,tropicana,GradientBoostingRegressor,sklearn.ensemble.gradient_boosting.GradientBoo...,Zack Soenen,tropicana_1013_job,1013,6dfcfdce-1bcc-477c-ba46-3e73fd2043fb


In [19]:
# Get best version of the registered
mlflow_client = mlflow.MlflowClient()
experiment_names=["many-models-parallel-training-job-mlflow30"]

experiment_ids = list(map(lambda x: mlflow_client.get_experiment_by_name(x).experiment_id, experiment_names))

runs = mlflow_client.search_runs(experiment_ids = experiment_ids,
                               filter_string=f"tags.brand = 'dominicks' AND tags.store = '1000' AND attributes.status != 'FAILED'")

best_metric = -1
best_run = None
for r in runs:
    if best_metric == -1:
        best_metric = r.data.metrics['test_rmse']
        best_run = r.info.run_id
    else:
        if r.data.metrics['test_rmse'] < best_metric:
            best_metric = r.data.metrics['test_rmse']
            best_run = r.info.run_id

print(best_run)

model = mlflow_client.search_model_versions(filter_string=f"run_id = '{best_run}'")
print(model[0].version)

65dcfeea-aa57-4c83-80a4-6b1ee9ac8c9c
18


In [46]:
# This doesn't work for some reason
ml_client.jobs.get(name="938a96c3-ab73-42a5-8c39-f16a3ce0d169")

CredentialUnavailableError: Failed to invoke the Azure CLI